In [ ]:
mount='/content/gdrive'
drive_root = mount + "/My Drive/Colab Notebooks/08_MIAR/actividades/proyecto practico"
from google.colab import drive
IN_COLAB=True

# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: making sure  /content/gdrive/My Drive/Colab Notebooks/08_MIAR/actividades/proyecto practico  exists.

Colab: Changing directory to  /content/gdrive/My Drive/Colab Notebooks/08_MIAR/actividades/proyecto practico
/content/gdrive/My Drive/Colab Notebooks/08_MIAR/actividades/proyecto practico
Archivos en el directorio: 
['dqn_SpaceInvaders-v0_weights_100000.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights_100000.h5f.index', 'dqn_SpaceInvaders-v0_weights_200000.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights_200000.h5f.index', 'dqn_SpaceInvaders-v0_weights_300000.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights_300000.h5f.index', 'dqn_SpaceInvaders-v0_weights_400000.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights_400000.h5f.index', 'dqn_SpaceInvaders-v0_weights_500000.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights_500000.h5f.index', 'dqn_SpaceInvader

In [ ]:
%pip install gym==0.17.3
%pip install git+https://github.com/Kojoley/atari-py.git
%pip install keras-rl2==1.0.5
%pip install tensorflow==2.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.9 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654617 sha256=91bc32397476350a4913e77f5e37451aa8b0a8e29f7c72b030f66dc51ad8ec5d
  Stored in directory: /root/.cache/pip/wheels/07/8b/b7/570cb90b10f17e85ccb291ba1f04af41ec697745104a2263eb
Successfully built gym
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 3.1.1
    Uninstalling cloudpickle-3.1.1:
      Successfully uninstalled cloudpickle-3.1.1
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2024.12.1 requires cloudpickle>

In [ ]:
from __future__ import division

import numpy as np
import gym
import cv2
import base64
import imageio
from PIL import Image
from collections import deque
from IPython.display import HTML

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Permute, BatchNormalization
from tensorflow.keras.optimizers.legacy import Adam

# === CONFIGURACIÓN ===
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
VIDEO_PATH = "/content/gdrive/My Drive/Colab Notebooks/08_MIAR/actividades/proyecto practico/space_invaders.mp4"
MODEL_PATH = "/content/gdrive/My Drive/Colab Notebooks/08_MIAR/actividades/proyecto practico/dqn3_SpaceInvaders-v0_weights_100000.h5f"

# === ENTORNO ===
env = gym.make("SpaceInvaders-v0")
nb_actions = env.action_space.n

# === MODELO ===
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=(WINDOW_LENGTH,) + INPUT_SHAPE))  # (4, 84, 84)
model.add(Conv2D(32, (8, 8), strides=(4, 4)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

# === CARGAR PESOS ===
model.load_weights(MODEL_PATH)

# === PREPROCESAMIENTO ===
def preprocess(obs):
    if obs.ndim == 3 and obs.shape[2] == 3:
        obs = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
    obs = cv2.resize(obs, (84, 84))
    obs = obs / 255.0
    return obs

# === INICIALIZACIÓN ===
obs = env.reset()
obs = obs[0] if isinstance(obs, tuple) else obs

frame_stack = deque(maxlen=WINDOW_LENGTH)
for _ in range(WINDOW_LENGTH):
    frame_stack.append(preprocess(obs))

done = False
frames = []

# === SIMULACIÓN ===
while not done:
    state = np.stack(frame_stack, axis=0)           # (4, 84, 84)
    state = np.expand_dims(state, axis=0)           # (1, 4, 84, 84)

    q_values = model.predict(state, verbose=0)
    action = np.argmax(q_values)

    obs, reward, done, info = env.step(action)
    frame_stack.append(preprocess(obs))

    frame = env.render(mode="rgb_array")
    frames.append(frame)

env.close()

# === GUARDAR VIDEO ===
imageio.mimsave(VIDEO_PATH, frames, fps=30)

# === MOSTRAR VIDEO EN EL NOTEBOOK ===
with open(VIDEO_PATH,'rb') as f:
    mp4 = f.read()
data_url = "data:video/mp4;base64," + base64.b64encode(mp4).decode()
display(HTML(f'<video width=400 controls><source src="{data_url}" type="video/mp4"></video>'))


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
